# Discretization

Now we demonstrate how to discretize an equation object.

Pre-coded sample objects are stored in `sample` attribute of `phyem` packages. Invoke these samples by

In [1]:
import sys
path_to_phyem = '../../../../'   # the path to dir that containing the phyem package.
sys.path.append(path_to_phyem)
import phyem as ph  # import the phyem package
ph.config._set_matplot_block(False)
samples = ph.samples

The partial differential equations of the canocical linear por-Hamiltonian are pre-coded. Call it through

In [2]:
oph = samples.pde_canonical_pH(n=3, p=3, periodic=False)[0]  # where o on `oph` means outer

Check `oph`, *outer oriented port-Hamiltonian*, with

In [3]:
oph.pr()

<Figure size 800x600 with 1 Axes>

We can take out the knowns and label them by

In [4]:
a3, b2 = oph.unknowns

We now test `oph` with test functions from the spaces where `a3` and `b2` come from, and label the test functions by $v^3$ and $u^2$,

In [5]:
wf = oph.test_with(oph.unknowns, sym_repr=[r'v^3', r'u^2'])
wf.pr()

<Figure size 1200x600 with 1 Axes>

Now, we apply integration by parts to the term indexed by `'1-1'`.

In [6]:
wf = wf.derive.integration_by_parts('1-1')
wf.pr()

<Figure size 1200x600 with 1 Axes>

We now apply a particular discretization to this weak formulation,

In [7]:
td = wf.td
td.set_time_sequence()  # initialize a time sequence
td.define_abstract_time_instants('k-1', 'k-1/2', 'k')
td.differentiate('0-0', 'k-1', 'k')
td.average('0-1', b2, ['k-1', 'k'])

td.differentiate('1-0', 'k-1', 'k')
td.average('1-1', a3, ['k-1', 'k'])
td.average('1-2', a3, ['k-1/2'])
dt = td.time_sequence.make_time_interval('k-1', 'k')

wf = td()
wf.pr()

wf.unknowns = [
    a3 @ td.time_sequence['k'],
    b2 @ td.time_sequence['k'],
]

wf = wf.derive.split(
    '0-0', 'f0',
    [a3 @ td.ts['k'], a3 @ td.ts['k-1']],
    ['+', '-'],
    factors=[1/dt, 1/dt],
)
wf.pr()

wf = wf.derive.split(
    '0-2', 'f0',
    [ph.d(b2 @ td.ts['k-1']), ph.d(b2 @ td.ts['k'])],
    ['+', '+'],
    factors=[1/2, 1/2],
)
wf.pr()

wf = wf.derive.split(
    '1-0', 'f0',
    [b2 @ td.ts['k'], b2 @ td.ts['k-1']],
    ['+', '-'],
    factors=[1/dt, 1/dt]
)
wf.pr()

wf = wf.derive.split(
    '1-2', 'f0',
    [a3 @ td.ts['k-1'], a3 @ td.ts['k']],
    ['+', '+'],
    factors=[1/2, 1/2],
)
wf.pr()

wf = wf.derive.rearrange(
    {
        0: '0, 3 = 2, 1',
        1: '3, 0 = 2, 1, 4',
    }
)
wf.pr()

<Figure size 1200x600 with 1 Axes>

We now can write the weak formulation with matrix proxies. Before doing that, we need to 

In [8]:
ph.space.finite(3)
mp = wf.mp()
mp.pr()

<Figure size 1200x800 with 1 Axes>

The matrix format of the weak formulation lead to a linear system,

In [9]:
ls = mp.ls()
ls.pr()

<Figure size 1200x800 with 1 Axes>

Note that, till this moment, everything is still abstract. To do the numerical simulation, we need to bring them to a particular implementation, for example, the `msepy`, standing for *mimetic spectral elements python*, by calling `ph.fem.apply` function,

In [10]:
msepy, obj = ph.fem.apply('msepy', locals())
print(msepy)
print(obj)

<module 'msepy.main' from 'C:\\Users\\Yi Zhang\\OneDrive\\Codes\\phyem\\web\\source\\jupyter\\../../../..\\phyem\\msepy\\main.py'>
{'a3': <MsePy Form \widehat{\alpha}^3 at 0x000001F0065F3690>, 'b2': <MsePy Form \widehat{\beta}^2 at 0x000001F07EC6C710>, 'ls': <msepy.tools.linear_system.dynamic.MsePyDynamicLinearSystem object at 0x000001F07EC8C090>}


`ph.fem.apply` takes two arguments, this first one indicates which particular implementation we want to use (in this case, we use `msepy`, the pure python mimetic spectral element implementation) and the second argument says what we want to take to this implementation. We use `locals()` which says everything in the local environment. We can customize it of course to move ahead with only a ecific set local variables.